In [ ]:
#!pip install catboost
#!pip install xgboost

In [ ]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Modelling
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

In [ ]:
df_original=pd.read_csv('/content/cars24_20221210.csv')
df=df_original.copy()

In [ ]:
df.head()

,name,make,model,city,year,fueltype,kilometerdriven,ownernumber,transmission,bodytype,storename,isc24assured,registrationcity,registrationstate,benefits,discountprice,price,url,createdDate
0,Hyundai Elite i20,Hyundai,Elite I20,Kochi,2017,Petrol,35939,2,Manual,Hatchback,Kochi-Pns,False,Kozhikode,Kerala,13898,47399,607000,https://www.cars24.com/buy-used-hyundai-elite-...,2022-03-24
1,Maruti Alto 800,Maruti,Alto 800,Mumbai,2016,Petrol + Cng,6180,1,Manual,Hatchback,Hi-West Mumbai,False,Mumbai,Maharashtra,12542,2099,326000,https://www.cars24.com/buy-used-maruti-alto-80...,2022-11-09
2,Maruti Dzire,Maruti,Dzire,Lucknow,2019,Petrol + Cng,46808,1,Manual,Sedan,Vibhuti Khand - Gomti Nagar,False,Lucknow,Uttar Pradesh,13898,26000,640000,https://www.cars24.com/buy-used-maruti-dzire-2...,2022-09-30
3,KIA SELTOS,Kia,Seltos,Bangalore,2020,Diesel,26045,1,Manual,SUV,Bengaluru-Pns,False,Bengaluru,Karnataka,9688,143700,1354000,https://www.cars24.com/buy-used-kia-seltos-202...,2022-07-04
4,Maruti Ertiga,Maruti,Ertiga,Kolkata,2017,Petrol,53056,1,Manual,SUV,Hi-Kolkata,False,Barrackpore,West Bengal,13898,22800,676000,https://www.cars24.com/buy-used-maruti-ertiga-...,2022-10-28


In [ ]:
df['year'].value_counts()

,count
year,
2018,1031
2019,996
2017,846
2020,774
2021,690
2016,572
2015,391
2014,377
2013,239


In [ ]:
from datetime import datetime
current_year=datetime.now().year
df['no_year']=current_year-df['year']

In [ ]:
df.head()

,name,make,model,city,year,fueltype,kilometerdriven,ownernumber,transmission,bodytype,storename,isc24assured,registrationcity,registrationstate,benefits,discountprice,price,url,createdDate,no_year
0,Hyundai Elite i20,Hyundai,Elite I20,Kochi,2017,Petrol,35939,2,Manual,Hatchback,Kochi-Pns,False,Kozhikode,Kerala,13898,47399,607000,https://www.cars24.com/buy-used-hyundai-elite-...,2022-03-24,8
1,Maruti Alto 800,Maruti,Alto 800,Mumbai,2016,Petrol + Cng,6180,1,Manual,Hatchback,Hi-West Mumbai,False,Mumbai,Maharashtra,12542,2099,326000,https://www.cars24.com/buy-used-maruti-alto-80...,2022-11-09,9
2,Maruti Dzire,Maruti,Dzire,Lucknow,2019,Petrol + Cng,46808,1,Manual,Sedan,Vibhuti Khand - Gomti Nagar,False,Lucknow,Uttar Pradesh,13898,26000,640000,https://www.cars24.com/buy-used-maruti-dzire-2...,2022-09-30,6
3,KIA SELTOS,Kia,Seltos,Bangalore,2020,Diesel,26045,1,Manual,SUV,Bengaluru-Pns,False,Bengaluru,Karnataka,9688,143700,1354000,https://www.cars24.com/buy-used-kia-seltos-202...,2022-07-04,5
4,Maruti Ertiga,Maruti,Ertiga,Kolkata,2017,Petrol,53056,1,Manual,SUV,Hi-Kolkata,False,Barrackpore,West Bengal,13898,22800,676000,https://www.cars24.com/buy-used-maruti-ertiga-...,2022-10-28,8


In [ ]:
df['discountprice'] = df['discountprice'].apply(lambda x: 0 if x < 0 else x)

In [ ]:
df['price_after_discount']=df['price']-df['discountprice']

In [ ]:
df.drop(columns=['year','price','discountprice'], inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()


In [ ]:
df['fueltype']=le.fit_transform(df['fueltype'])
df['transmission']=le.fit_transform(df['transmission'])
df['bodytype']=le.fit_transform(df['bodytype'])
df['isc24assured']=le.fit_transform(df['isc24assured'])
df['registrationstate']=le.fit_transform(df['registrationstate'])

In [ ]:
df['fueltype']=df['fueltype'].astype(int)
df['transmission']=df['transmission'].astype(int)
df['bodytype']=df['bodytype'].astype(int)
df['isc24assured']=df['isc24assured'].astype(int)
df['registrationstate']=df['registrationstate'].astype(int)


In [ ]:
df.drop(columns=['city','storename','benefits','registrationcity','url','createdDate'], inplace=True)

In [ ]:
df.head()

,name,make,model,fueltype,kilometerdriven,ownernumber,transmission,bodytype,isc24assured,registrationstate,no_year,price_after_discount
0,Hyundai Elite i20,Hyundai,Elite I20,1,35939,2,1,0,0,7,8,559601
1,Maruti Alto 800,Maruti,Alto 800,2,6180,1,1,0,0,9,9,323901
2,Maruti Dzire,Maruti,Dzire,2,46808,1,1,5,0,14,6,614000
3,KIA SELTOS,Kia,Seltos,0,26045,1,1,4,0,6,5,1210300
4,Maruti Ertiga,Maruti,Ertiga,1,53056,1,1,4,0,15,8,653200


In [ ]:
df['make_model_encoded']=df['make']+df['model']

In [ ]:
df.drop(columns=['name','make','model'],inplace=True)

In [ ]:
df['make_model_encoded']=le.fit_transform(df['make_model_encoded'])

In [ ]:
df['make_model_encoded']=df['make_model_encoded'].astype(int)

In [ ]:
df.head()

,fueltype,kilometerdriven,ownernumber,transmission,bodytype,isc24assured,registrationstate,no_year,price_after_discount,make_model_encoded
0,1,35939,2,1,0,0,7,8,559601,29
1,2,6180,1,1,0,0,9,9,323901,64
2,2,46808,1,1,5,0,14,6,614000,71
3,0,26045,1,1,4,0,6,5,1210300,49
4,1,53056,1,1,4,0,15,8,653200,73


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = df.drop('price_after_discount', axis=1)
y = df['price_after_discount']
X_scaled = scaler.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X_scaled,y,test_size=0.2,random_state=42)

In [ ]:
print("X trained shape values:",X_train.shape)
print("X test shape values:",X_test.shape)
print("y trained shape values:",y_train.shape)
print("y test shape values:",y_test.shape)

X trained shape values: (5149, 9)
X test shape values: (1288, 9)
y trained shape values: (5149,)
y test shape values: (1288,)


In [ ]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [ ]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(),
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor(),
    "SVR": SVR()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)


    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')

    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)

    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 218727.9118
- Mean Absolute Error: 158094.0794
- R2 Score: 0.5550
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 209400.1649
- Mean Absolute Error: 152141.8489
- R2 Score: 0.5657


Lasso
Model performance for Training set
- Root Mean Squared Error: 218727.9118
- Mean Absolute Error: 158093.8934
- R2 Score: 0.5550
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 209400.1024
- Mean Absolute Error: 152141.6273
- R2 Score: 0.5657


Ridge
Model performance for Training set
- Root Mean Squared Error: 218727.9155
- Mean Absolute Error: 158088.5390
- R2 Score: 0.5550
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 209399.5580
- Mean Absolute Error: 152135.6490
- R2 Score: 0.5657


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 148829.7243
- Mean 

In [ ]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
7,CatBoosting Regressor,0.900558
6,XGBRegressor,0.890618
5,Random Forest Regressor,0.870086
4,Decision Tree,0.761439
3,K-Neighbors Regressor,0.685627
2,Ridge,0.565677
1,Lasso,0.565675
0,Linear Regression,0.565674
8,AdaBoost Regressor,0.517907
9,SVR,-0.066877


In [ ]:
#Initialize few parameter for Hyperparamter tuning

rf_params = {"max_depth": [5, 8, 15, None, 10],
             "max_features": [5, 7, "auto", 8],
             "min_samples_split": [2, 8, 15, 20],
             "n_estimators": [100, 200, 500, 1000]}

xgboost_params = {"learning_rate": [0.1, 0.01],
                  "max_depth": [5, 8, 12, 20, 30],
                  "n_estimators": [100, 200, 300],
                  "colsample_bytree": [0.5, 0.8, 1, 0.3, 0.4]}
catboost_params={
    'learning_rate': [0.01, 0.05, 0.1, 0.3],
    'depth': [4, 6, 8, 10],
    'l2_leaf_reg': [1, 3, 5, 7, 9],
    'iterations': [100, 300, 500, 800],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bylevel': [0.5, 0.8, 1.0],
    'bagging_temperature': [0, 1, 5]}

Decision_params={
    'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 20, 30],
}

In [ ]:
# Models list for Hyperparameter tuning
randomcv_models = [
                   ("RF", RandomForestRegressor(), rf_params),
                   ("XGboost",XGBRegressor(),xgboost_params),
                   ("Catboost",CatBoostRegressor(verbose=False),catboost_params),
                   ("Decision Tree",DecisionTreeRegressor(),Decision_params)

                   ]

In [ ]:
##Hyperparameter Tuning
from sklearn.model_selection import RandomizedSearchCV

model_param = {}
for name, model, params in randomcv_models:
    random = RandomizedSearchCV(estimator=model,
                                   param_distributions=params,
                                   n_iter=100,
                                   cv=3,
                                   verbose=2,
                                   n_jobs=-1)
    random.fit(X_train, y_train)
    model_param[name] = random.best_params_

for model_name in model_param:
    print(f"---------------- Best Params for {model_name} -------------------")
    print(model_param[model_name])

Fitting 3 folds for each of 100 candidates, totalling 300 fits


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
69 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
69 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.11/dist-packages/sklearn/utils/

Fitting 3 folds for each of 100 candidates, totalling 300 fits


/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Fitting 3 folds for each of 100 candidates, totalling 300 fits


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 32 is smaller than n_iter=100. Running 32 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 3 folds for each of 32 candidates, totalling 96 fits
---------------- Best Params for RF -------------------
{'n_estimators': 1000, 'min_samples_split': 2, 'max_features': 7, 'max_depth': None}
---------------- Best Params for XGboost -------------------
{'n_estimators': 300, 'max_depth': 8, 'learning_rate': 0.1, 'colsample_bytree': 0.4}
---------------- Best Params for Catboost -------------------
{'subsample': 0.6, 'learning_rate': 0.3, 'l2_leaf_reg': 5, 'iterations': 500, 'depth': 4, 'colsample_bylevel': 1.0, 'bagging_temperature': 0}
---------------- Best Params for Decision Tree -------------------
{'splitter': 'best', 'max_depth': 10, 'criterion': 'poisson'}


In [ ]:
## Retraining the models with best parameters
models = {
    "Random Forest Regressor": RandomForestRegressor(n_estimators=200, min_samples_split=2, max_features=5, max_depth=None,
                                                     n_jobs=-1),
     "Xgboost Regressor":XGBRegressor(n_estimators= 300,learning_rate=0.1,
                                     max_depth=5,colsample_bytree=0.5),
    "Catboost Regressor": CatBoostRegressor(depth=8, l2_leaf_reg=3, learning_rate=0.1),
    "Decision Tree":DecisionTreeRegressor(criterion='squared_error',splitter='best',max_depth=None)


}
model_list1=[]
r2_list2=[]
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print(list(models.keys())[i])
    model_list1.append(list(models.keys())[i])

    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')

    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list2.append(model_test_r2)

    print('='*35)
    print('\n')

Random Forest Regressor
Model performance for Training set
- Root Mean Squared Error: 43802.2709
- Mean Absolute Error: 27250.1928
- R2 Score: 0.9822
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 118544.6503
- Mean Absolute Error: 73925.4544
- R2 Score: 0.8608


Xgboost Regressor
Model performance for Training set
- Root Mean Squared Error: 69040.4542
- Mean Absolute Error: 47813.5391
- R2 Score: 0.9557
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 100630.3283
- Mean Absolute Error: 65624.2656
- R2 Score: 0.8997


0:	learn: 307570.6636316	total: 3.98ms	remaining: 3.98s
1:	learn: 290776.8354642	total: 6.67ms	remaining: 3.33s
2:	learn: 274887.8580691	total: 9.39ms	remaining: 3.12s
3:	learn: 260562.2254677	total: 12ms	remaining: 2.98s
4:	learn: 249148.8368226	total: 13.5ms	remaining: 2.68s
5:	learn: 238996.8707498	total: 16.2ms	remaining: 2.68s
6:	learn: 228943.1377844	total: 18.8ms	remaining: 2.

In [ ]:
pd.DataFrame(list(zip(model_list1, r2_list2)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
2,Catboost Regressor,0.908341
1,Xgboost Regressor,0.899696
0,Random Forest Regressor,0.860804
3,Decision Tree,0.764214
